In [2]:
!pip install transformers pandas tqdm


In [3]:
import pandas as pd
from transformers import MarianTokenizer, MarianMTModel
from tqdm import tqdm

# Load model and tokenizer for Bengali → English
model_name = "Helsinki-NLP/opus-mt-bn-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Load your Bengali text CSV (change filename and encoding if needed)
df = pd.read_csv("/kaggle/input/bengali-text-2/Bengali_text.csv", encoding='utf-8-sig').head(10)  # Column: 'bengali_text'

# Translate function
def translate_bn_to_en(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    batch = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt")
    gen = model.generate(**batch, max_new_tokens=100)
    return tokenizer.decode(gen[0], skip_special_tokens=True)

# Apply translation with progress bar
tqdm.pandas()
df["translated_english"] = df["Captions"].progress_apply(translate_bn_to_en)

# Save output
df.to_csv("translated_bengali_10rows.csv", index=False)
print("✅ Bengali to English translation complete. Saved to translated_bengali_10rows.csv")


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4079: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
100%|██████████| 10/10 [00:08<00:00,  1.13it/s]

✅ Bengali to English translation complete. Saved to translated_bengali_10rows.csv


In [6]:
import pandas as pd
import re
from transformers import MarianMTModel, MarianTokenizer

# Bengali to English model and tokenizer
model_name = "Helsinki-NLP/opus-mt-bn-en"
tokenizer_bn_en = MarianTokenizer.from_pretrained(model_name)
model_bn_en = MarianMTModel.from_pretrained(model_name)

# Function to detect Bengali script using Unicode
bengali_pattern = re.compile(r'[\u0980-\u09FF]')

def contains_bengali(text):
    return bool(bengali_pattern.search(text))

# Function to translate Bengali to English
def translate_bn_to_en(text):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    batch = tokenizer_bn_en.prepare_seq2seq_batch([text], return_tensors="pt")
    gen = model_bn_en.generate(**batch)
    return tokenizer_bn_en.decode(gen[0], skip_special_tokens=True)

# Final logic to translate only if Bengali detected
def smart_translate(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    if contains_bengali(text):
        return translate_bn_to_en(text)
    return text

# Load your CSV file (change path to your actual file path)
file_path = "/kaggle/input/bengali-text-2/Bengali_text.csv"  # 📝 update this
df = pd.read_csv(file_path, encoding="utf-8")

# Process first 10 rows only for testing
df_subset = df.head(10).copy()

# Change 'text_column_name' to the actual name in your CSV (e.g., 'text')
df_subset['translated_text'] = df_subset['Captions'].apply(smart_translate)

# Save output to new CSV (optional)
df_subset.to_csv("translated_output.csv", index=False)

# Display the result
print(df_subset[['Captions', 'translated_text']])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4079: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


                                            Captions  \
0  আম্মাঃ HSC চলে আসছে , এখন থেকে তোর মোবাইল , ল্...   
1  WHEN YOUR COUSINS TAKES YOU TO THE DHAN KHET A...   
2  WHEN HE SAID 10 MINUTES BUT IT WAS ONLY 2 MINUTES   
3  SHE - I CAN'T BE WITH YOU   -তবে শেষবারের মত দ...   
4  যখন কোন Teacher বলে   "সত্যটা বলো, তাহলে আর কি...   
5  আয়মান সাদিক    ব্যয়মান সাদিক    পরমমান সাদিক  ...   
6  When your younger brothers asks  "ভাইয়া আমরা ক...   
7  WHEN YOU ARE ABOUT TO BANG HER & SHE SAYS দাঁড়...   
8  BIOLOGY THEORY IS IMPORTANT BUT, BIOLOGY PRACT...   
9  Nije kono content Create kora    Onno ke hoga ...   

                                     translated_text  
0  Amma: HC is coming back, your mobile, laptop's...  
1  WHEN YOUR COUSINS TAKES YOU TO THE DHAN KHET A...  
2  WHEN HE SAID 10 MINUTES BUT IT WAS ONLY 2 MINUTES  
3           SHE-ICAT: Let's see it for the last time  
4  When you say something, "Speak the truth, then...  
5                      Ayman Sadýk Damak Sadýk Sadýk 

# Best among others on 10 rows

In [7]:
!pip install transformers sentencepiece pandas langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.8 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=fce52e5074bd7df602196ea7ca26d8783565f2c06e9c666842a517994b2080fa
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [9]:
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from langdetect import detect

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name)

# Load your dataset (update with your CSV file)
df = pd.read_csv("/kaggle/input/bengali-text-2/Bengali_text.csv", encoding='utf-8')

# Work with just first 10 rows for now
sample_df = df.head(10)

def detect_and_translate(text):
    try:
        lang = detect(text)
        if lang not in tokenizer.lang_code_to_id:
            lang = "bn"  # fallback for Bengali/Hinglish mix
        tokenizer.src_lang = lang

        encoded = tokenizer(text, return_tensors="pt")
        generated = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en"])
        translated = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]
        return translated
    except Exception as e:
        print(f"Translation failed for: {text}")
        return text

# Add translation column
sample_df["translated_text"] = sample_df["Captions"].apply(detect_and_translate)

# # Change 'text_column_name' to the actual name in your CSV (e.g., 'text')
# df_subset['translated_text'] = df_subset['Captions'].apply(smart_translate)

# Save output to new CSV (optional)
sample_df.to_csv("translated_output_m2m.csv", index=False)

# Show result
print(sample_df[["Captions", "translated_text"]])


                                            Captions  \
0  আম্মাঃ HSC চলে আসছে , এখন থেকে তোর মোবাইল , ল্...   
1  WHEN YOUR COUSINS TAKES YOU TO THE DHAN KHET A...   
2  WHEN HE SAID 10 MINUTES BUT IT WAS ONLY 2 MINUTES   
3  SHE - I CAN'T BE WITH YOU   -তবে শেষবারের মত দ...   
4  যখন কোন Teacher বলে   "সত্যটা বলো, তাহলে আর কি...   
5  আয়মান সাদিক    ব্যয়মান সাদিক    পরমমান সাদিক  ...   
6  When your younger brothers asks  "ভাইয়া আমরা ক...   
7  WHEN YOU ARE ABOUT TO BANG HER & SHE SAYS দাঁড়...   
8  BIOLOGY THEORY IS IMPORTANT BUT, BIOLOGY PRACT...   
9  Nije kono content Create kora    Onno ke hoga ...   

                                     translated_text  
0  Mom: HSC is coming, from now on your mobile, l...  
1  When your cousins touch you to the DHAN KHET a...  
2    When he left 10 minutes, it was just 2 minutes.  
3  SHE - I can't be with you - but let it look li...  
4  When a teacher says, “Tell the truth, I’ll say...  
5    Sadiq Sadiq Sadiq Sadiq Sadiq Sadiq Sadiq Sadiq 

<ipython-input-9-2b76079c7045>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df["translated_text"] = sample_df["Captions"].apply(detect_and_translate)
